---

6.1 Define the task
the task is to traing a ai modle to predict what difrent fasion items are 

is will predict what 10 types of fasion items as are by trainging to a nural network on prexisting data and then give it new data 



<h1>66.1.1 Frame the problem</h1>

<!-- hat are you trying to predict?  -->
i will start of by doing some reserch on how to predict data using a prexisting data set call ed mnist 

then in will us the reserch that i learned from predicting numbers and apply the same tecneq to predict fasion  




<!-- And is the data (samples and labels) available? -->

<!-- What is the type of the problem?  -->

<!-- Binary classification, single label multiclass, multilabel multiclass, regression?  -->

<!-- Hypotheses -->

<!-- - the outputs can be predicted from the inputs; -->
<!-- - the available data is sufficiently informative. -->

The hypotheses might not be met even if your problem is well defined.

For example, an attempt to the predict stock prices from the recent past will likely fail because recent prices contain very little predictive information.

### Nonstationariness

Nonstationary problems have a time ordering.

For example, a clothing recommender will have to take the time of year into account (swimsuits in summer, coats in winter).

Either gather information from a period when the problem is stationary (a succession of summers) or train the model on recent trends (the past few weeks) or on all data but include the time of year as an input.

(In the industry this means a lot of retraining and finetuning all the time.)

A third hypothesis: **the future is like the past**.

The kind of machine learning we study now **only spots patterns in collected data** – data that lies in the past.

We assume, when we use a trained model, that the past is relevant today.

---

## 6.1.2 Collect a dataset

### Investing in data annotation infrastructure

This is more for companies: should you pay people to annotate your data?


Fun (?) fact: nobody really knows how many people do this, but comapnies like OpenAI use *armies* of very low-paid workers to annotate and check data. Fairly ironic to think that so many humans are working *for* the AI systems, as it were, even as those systems are supposed to be working *for us*...

### Beware of non-representative data

You should strive to make sure that your **training data** and the **real world (unseen)** data you're using your model on come from the **same distribution**.

#### Examples

Very clean and well-lit pictures in training, real-world pictures from social media in production, you're in for trouble!

Personal issue: trained on literary corpora for a bot interacting with actors. Humans would write fast, make mistakes, etc., in a way that wasn't at all "literary"!

*Concept drift*: much of the data in the world changes constantly. Chollet's example: recommender system, you can't have the same system now as in a few years ago!

#### The problem of sampling bias

If you based your estimation of who won the election that night just on this one newspaper, you would be wrong!
<!-- <img style="float:right;height:550px" src="images/chollet.sampling-bias.png"> -->
<img  style="float:right;height:550px" src="https://drive.google.com/uc?id=1CnnE3seK1nxxqa17-eoF1-PwIwHad7hg">

<small style="position:absolute;bottom:0;right:0">DLWP, p.184</small>

## 6.1.3 Understand your data

## 6.1.4 Choose a measure of success

### Workflow

1. **Define** the problem and **collect data** (samples and labels)
2. Choose the **success criterion** (**loss**) and monitoring **metrics**
3. Chose a **validation protocol** (hold-out...)
4. **Prepare your data**
5. Check your **baselines**
5. **Train**  
   a. a first underfitting model (still with statistical power, not a total wreck)  
   b. a larger overfitting model  
7. **Tune**  
   a. **Regularise** the overfitting model  
   b. **Tweak** hyperparameters based on performance on the **validation set**  
8. **Retrain on the entire training set** and evaluate on your **unseen test set**


### Experiments

- Different architectures (number of layers, units per layer)  
- Dropout
- L1/L2 regularisation
- Other hyperparameters...

## 6.1.3 Understand your data

## 6.1.4 Choose a measure of success

What is important? Accuracy? Precision? Something else?

The measure of success, whatever it is, will influence the choice of loss function.

You may have to design your own loss function – just remember that backpropagation demends smoothness.

Once your aims are clear, you have to decide how to measure progress as you tune your models.

---

## 6.2 Develop a model

## 6.2.1 Prepare the data

### Vectorisation 

- all inputs must be floating point tensors;
- for example, words in text data  are one-hot encoded.

### Value Normalisation

- digit classification – integer vector elements in $[0, 255]$ normalised by casting as floats and dividing by 255 to give floating point data in $[0, 1]$;
- house prices – the data was normalised so that each feature had mean of 0 and std of 1.

Gradient descent works best if data is homogeneous and constrained.

Constrain

- aim to put data values in $[0, 1]$ or $[-1, 1]$;
- weights and biases are initialised with small values otherwise a few large inputs might dominate training.

Homogeneity 

- all features take values in roughly the same range;
- inhomogeneity can trigger large gradient updates and hamper convergence.

Normalisation to a mean of 0 and standard deviation of 1 can help but is not always necessary;

(E.g. we didn't normalise the digit data in this way).

### Missing values

- sometimes a data point lacks a feature;
- you could fill the missing data with zeros (as long as 0 isn't already meaningful);
- the network will learn to ignore 0's because the 0's don't correlate with anything.

perhaps the test data has missing values but the training data does not.

In that case, duplicate some training data and randomly place 0's in some features (but leave the test data alone!) so that the network can learn to ignore missing values.

--

## 6.2.2 Choose an evaluation protocol

Our goal is to *generalise* well. For that, we need to measure how well we are doing outside training.

- Maintaining a **holdout validation set**: this is the way to go when you have plenty of data;
- Doing **K-fold cross-validation**: this is the right choice when you have too few samples for holdout validation to be reliable;
- Doing **iterated K-fold validation**: the most accurate, but also the most expensive, method when little data is available.


## 6.2.3 Beat a baseline

We aim for *statistical power* – a small model that can beat a common sense check

- for example, in digit classification, a baseline algorithm merely assigns labels at random – our small model must beat 10%
- For the movie reviews, our model must perform better than 50% accuracy

(Assuming that MNIST and IMDB have an equal distribution of class membership) 

ut sometimes, after much experimentation, we cannot beat the baseline.

Perhaps one or more of the hypotheses are false.

here is no prescription for model design, but there are guidelines.

- the smallest capacity model with statistical power;
- `rmsprop` optimiser with default learning rate;
- loss and last layer activation according to the table: 

//---------------------------------------

//---------------------------------------

//---------------------------------------

//---------------------------------------

//---------------------------------------

In [56]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [57]:
# MNIST

# load
from tensorflow.keras.datasets import mnist
from tensorflow.keras import optimizers
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# preprocess
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

from tensorflow.keras.utils import to_categorical 
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)



# build
from tensorflow.keras import models, layers
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28, )))
network.add(layers.Dense(10, activation='softmax'))
network.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)



# 0-1
# lr 0.001 - 0.00

class TestDat():
    def __init__(self, learning_rate, momentum,accuracy):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.accuracy = accuracy


TestArray = [None for c in range(9**2)]

# TestArray[0] = TestDat(0,1)
# TestArray[1] = TestDat(0,1)


# print (TestArray[0].Dat1) # this is Test1
# print (TestArray[1].Dat1) # this is Test2    

iterCount = 0

for i in range (1,9): # (initial, final but not included)
  for j in range (1,9): 
    print("test: ","learning rate" , (i/ 100) , "momentum", (j/ 10) , i , "." , j)

    # redefine the var network 
    network.compile(
      optimizer=optimizers.RMSprop(lr=(i/ 100), momentum=(j/ 10)),
      loss='categorical_crossentropy', 
      metrics=['accuracy']
    )
    # TestArray[0] = TestDat(0,1)
    # print(network.compile.metrics, "teeeeeeeeeeeeeeeeeeeeeee")
    # model.compile(optimizer=, loss=, metrics=['accuracy',...])
    # network.compile.evaluate()
    # print(network.compile())
    history = network.fit(train_images, train_labels, epochs=5, batch_size=128)
    # print(network.compile)
    # print(history.history['accuracy'][0], "weeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee")
    TestArray[iterCount] = TestDat((i/ 100),(j/ 10),history.history['accuracy'][3])
    # if(iterCount == 9^2):
    for spit in range (0,iterCount):
      print("learning_rate: " ,TestArray[spit].learning_rate, "momentum: " ,TestArray[spit].momentum, "accuracy: " ,TestArray[spit].accuracy)
    iterCount += 1
    # history.history['accuracy'][0], "weeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee"

# train


2022-11-30 12:08:49.106318: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-30 12:08:49.106348: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (splinter0666): /proc/driver/nvidia/version does not exist
2022-11-30 12:08:49.106707: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


test:  learning rate 0.01 momentum 0.1 1 . 1
Epoch 1/5
469/469 [==============================] - 1s 2ms/step - loss: 0.2518 - accuracy: 0.9266
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 0.1028 - accuracy: 0.9698
Epoch 3/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0675 - accuracy: 0.9797
Epoch 4/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0486 - accuracy: 0.9849
Epoch 5/5
 57/469 [==>...........................] - ETA: 0s - loss: 0.0372 - accuracy: 0.9890

KeyboardInterrupt: 

In [ ]:
word_file = "/usr/share/dict/words"
WORDS = open(word_file).read().splitlines()

#### The problem of sampling bias

If you based your estimation of who won the election that night just on this one newspaper, you would be wrong!
<!-- <img style="float:right;height:550px" src="images/chollet.sampling-bias.png"> -->
<img  style="float:right;height:550px" src="https://drive.google.com/uc?id=1CnnE3seK1nxxqa17-eoF1-PwIwHad7hg">

<small style="position:absolute;bottom:0;right:0">DLWP, p.184</small>

use comand in the terminal --> pip install random-word

please import this package so that you can generate the words and make the data set 

In [ ]:
# First pip install random-word
from random_word import RandomWords
r = RandomWords()

# Return a single random word
rand_word_incoding = r.get_random_word()

print(rand_word_incoding)
# Return list of Random words
# r.get_random_words()
# # Return Word of the day
# r.word_of_the_day()

# print(r)


sustenanceless


In [ ]:
import random
# randint generates a random integar between the first parameter and the second

# random.randint(1, 100)

import numpy as np

#--------------------------------------------------

categorys = ['top', 'trouser', 'pullover', 'dress',
             'coat', 'sandle', 'shirt', 'sneaker', 'bag', 'bnkle boot']



#--------------------------------------------------

def genItemDescription(selected_catecogory_num, descreption_of_listing, probablity_of_outcome):
    # selected_catecogory_num =  0
    # descreption_of_listing = []
    actual_outcome_counter = 0

    for i in range(20): # (initial, final but not included)
        probabelety = random.randint(1, 100)    
        if probabelety > probablity_of_outcome:
            descreption_of_listing.append(categorys[selected_catecogory_num])
            actual_outcome_counter += 1
        else:
            descreption_of_listing.append(r.get_random_word())

    #----------------------------------------------------------------

    descreption_of_listing.append(actual_outcome_counter)
    descreption_of_listing.append(probablity_of_outcome)
    print(descreption_of_listing) 

    #---------end of function----------------------------------------
    return descreption_of_listing

#--------------------------------------------------
descreption_of_listing = []
data_test_vectorizer = genItemDescription(1, descreption_of_listing, 80)

    # for i in range(10): # (initial, final but not included)

show_slice = data_test_vectorizer[:20]
print(show_slice)



# rechape_array = np.array(descreption_of_listing)

# print(random.randint(1, 100))


# label_batch[i].numpy()


['trouser', 'unbowered', 'springled', 'trouser', 'traversals', 'cleidorrhexis', 'trouser', 'counterraid', 'bullous', 'trouser', 'minimax', 'aloysia', 'hypernormal', 'trouser', 'trouser', 'trouser', 'chronologies', 'trouser', 'heptatomic', 'enhydrinae', 8, 80]
['trouser', 'unbowered', 'springled', 'trouser', 'traversals', 'cleidorrhexis', 'trouser', 'counterraid', 'bullous', 'trouser', 'minimax', 'aloysia', 'hypernormal', 'trouser', 'trouser', 'trouser', 'chronologies', 'trouser', 'heptatomic', 'enhydrinae']


In [ ]:
def str_to_ints(s):
    s_enc = s.encode()
    print(f"{list(s_enc)}") # encode into bytes, then turn into a list

[116, 114, 111, 117, 115, 101, 114]
['trouser', 'unbowered', 'springled', 'trouser', 'traversals', 'cleidorrhexis', 'trouser', 'counterraid', 'bullous', 'trouser', 'minimax', 'aloysia', 'hypernormal', 'trouser', 'trouser', 'trouser', 'chronologies', 'trouser', 'heptatomic', 'enhydrinae']


In [ ]:
class Vectorizer:
    """
    An implementation of the transformations in the above schema.
    """
    def standardize(self, text):
        """Make lowercase and remove punctuation"""
        text = text.lower()                                # Python built-in list of punctuation characters
        return "".join(char for char in text if char not in string.punctuation)

    def tokenize(self, text):
        """Standardize then split on space"""
        text = self.standardize(text)
        return text.split()

    def make_vocabulary(self, dataset):
        """Generate token vocabulary from text"""
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            tokens = self.tokenize(text) # this will standardize
            for token in tokens:                                  # first element will be 2
                if token not in self.vocabulary:                  # then len(vocab) is 3 →
                    self.vocabulary[token] = len(self.vocabulary) # next token is 3, etc.
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        """Encode string to tokens"""
        tokens = self.tokenize(text) # this will standardize
        return [self.vocabulary.get(token, 1) for token in tokens]

    def decode(self, int_sequence):
        """Decode tokens to string"""        
        return " ".join(
            self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)

    def one_hot_encode(self, encoded_sequence):
        vector = np.zeros((len(encoded_sequence), len(self.vocabulary)))
        for i, token in enumerate(encoded_sequence):
            vector[i, token] = 1
        return vector
    
    def one_hot_decode(self, one_hot_sequence):
        return list(np.where(one_hot_sequence == 1)[1])

In [61]:

# show_slice = data_test_vectorizer[:20]
# str_to_ints(show_slice[0])

# print(show_slice)

show_slice = data_test_vectorizer[:20]
str_to_ints(show_slice[0])

print(show_slice)

# show_slice

vectorizer = Vectorizer()
# dataset = [
#     "I write, erase, rewrite",
#     "Erase again, and then",
#     "A poppy blooms.",
# ]
vectorizer.make_vocabulary(show_slice)


# test_sentence = "I write, rewrite, and still rewrite again"
test_sentence = show_slice[0]



encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)


one_hot_sentence = vectorizer.one_hot_encode(encoded_sentence)
print(one_hot_sentence)




# ----------------------------------------------------------------------------------------------------------------------------
vectorizer1 = Vectorizer()
# dataset = [
#     "I write, erase, rewrite",
#     "Erase again, and then",
#     "A poppy blooms.",
# ]
vectorizer1.make_vocabulary(show_slice)


# test_sentence = "I write, rewrite, and still rewrite again"
test_sentence1 = show_slice[0]

# test?
# test2



encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)


one_hot_sentence = vectorizer.one_hot_encode(encoded_sentence)
print(one_hot_sentence)

# ----------------------------------------------------------------------------------------------------------------------------


# vectorizer = Vectorizer()
# dataset = [
#     "I write, erase, rewrite",
#     "Erase again, and then",
#     "A poppy blooms.",
# ]

# vectorizer.make_vocabulary(show_slice)


# test_sentence = "I write, rewrite, and still rewrite again"
# encoded_sentence = vectorizer.encode(test_sentence)
# print(encoded_sentence)


# one_hot_sentence = vectorizer.one_hot_encode(encoded_sentence)
# print(one_hot_sentence)


[116, 114, 111, 117, 115, 101, 114]
['trouser', 'unbowered', 'springled', 'trouser', 'traversals', 'cleidorrhexis', 'trouser', 'counterraid', 'bullous', 'trouser', 'minimax', 'aloysia', 'hypernormal', 'trouser', 'trouser', 'trouser', 'chronologies', 'trouser', 'heptatomic', 'enhydrinae']
[2]
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


2022-11-30 11:23:34.044289: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 11:23:34.888090: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-30 11:23:34.888135: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-11-30 11:23:34.888139: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

[72, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100, 33]
b'Hello, world!'


In [ ]:
import re
import string
import unidecode
import os, pathlib, shutil, random



import numpy as np
import tensorflow as tf



def str_to_ints(s):
    s_enc = s.encode()
    print(f"{s} | {s_enc} | {list(s_enc)}") # encode into bytes, then turn into a list

s = "Jérémie"
str_to_ints(s)
str_to_ints(s.lower())
str_to_ints(unidecode.unidecode(s.lower())) # removing 




s = "the quick brown fox jumps over the lazy dog"
s_split = s.split(" ")
print(s)
print(s_split)

# --
def n_grams(l, n=2):
    n_grams = []
    for i in range(0, len(s_split) - n + 1):
        n_grams.append(s_split[i:i+n])
    return n_grams

#--
n_grams(s_split, n=2) # each token is two words


n_grams(s_split, n=3) # each token is three words


vocab = {token:i for i,token in enumerate(sorted(list(set(s_split))))}


Jérémie | b'J\xc3\xa9r\xc3\xa9mie' | [74, 195, 169, 114, 195, 169, 109, 105, 101]
jérémie | b'j\xc3\xa9r\xc3\xa9mie' | [106, 195, 169, 114, 195, 169, 109, 105, 101]
jeremie | b'jeremie' | [106, 101, 114, 101, 109, 105, 101]
the quick brown fox jumps over the lazy dog
['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

class sklearn.preprocessing.OneHotEncoder(*, categories='auto', drop=None, sparse=True, dtype=<class 'numpy.float64'>, handle_unknown='error', min_frequency=None, max_categories=None)[source]¶


# #--------------------------------------------------
# # jimjam = tf.one_hot(
# #     indices = data_test_vectorizer,
# #     depth = 10,
# #     on_value=None,w
# #     off_value=None,
# #     axis=None,
# #     dtype=None,
# #     name=None
# # )


# tf.keras.layers.experimental.preprocessing.TextVectorization(
#     max_tokens=100,
#     standardize="lower_and_strip_punctuation",
#     split="whitespace",
#     ngrams=None,
#     output_mode="int",
#     output_sequence_length=None,
#     pad_to_max_tokens=False,
#     vocabulary=None,
# )


# print(jimjam)



# define example
data = ['cold', 'cold', 'warm', 'cold', 'hot',
        'hot', 'warm', 'cold', 'warm', 'hot']

values = np.array(data)

# first apply label encoding
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

# now we can apply one hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
 
tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=None,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    ngrams=None,
    output_mode="int",
    output_sequence_length=None,
    pad_to_max_tokens=False,
    vocabulary=None,
    **kwargs
)
 
 
 
 
 
 text_dataset = tf.data.Dataset.from_tensor_slices(["foo", "bar", "baz"])
 max_features = 5000  # Maximum vocab size.
 max_len = 4  # Sequence length to pad the outputs to.
 embedding_dims = 2

  # Create the layer.
#  vectorize_layer = TextVectorizatio...  max_tokens=max_feature...  output_mode='int',
# output_sequence_length=max_len)

vocab_data = ["earth", "wind", "and", "fire"]
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=max_len,
    vocabulary=vocab_data
)




  # Now that the vocab layer has been created, call `adapt` on the text-only
  # dataset to create the vocabulary. You don't have to batch, but for large
  # datasets this means we're not keeping spare copies of the dataset.
 vectorize_layer.adapt(text_dataset.batch(64))

  # Create the model that uses the vectorize text layer
 model = tf.keras.models.Sequential()

  # Start by creating an explicit input layer. It needs to have a shape of
  # (1,) (because we need to guarantee that there is exactly one string
  # input per batch), and the dtype needs to be 'string'.
 model.add(tf.keras.Input(shape=(1,), dtype=tf.string))

  # The first layer in our model is the vectorization layer. After this
  # layer, we have a tensor of shape (batch_size, max_len) containing vocab
  # indices.
 model.add(vectorize_layer)

  # Now, the model can map strings to integers, and you can add an embedding
  # layer to map these integers to learned embeddings.
 input_data = [["foo qux bar"], ["qux baz"]]
 model.predict(input_data)
    #  array([[2, 1, 4, 0   [1, 3, 0, 0]])


In [ ]:
vocab_data = ["earth", "wind", "and", "fire"]
>> > max_len = 4  # Sequence length to pad the outputs to.
>> >
>> >  # Create the layer, passing the vocab directly. You can also pass the
>> >  # vocabulary arg a path to a file containing one vocabulary word per
>> >  # line.
>> > vectorize_layer = TextVectorization(
>> >     max_tokens=max_features,
    ...  max_tokens=max_features,
    ...  output_mode='int',
    ...  output_sequence_length=max_len,
    ...  vocabulary=vocab_data)
>> >
>> >  # Because we've passed the vocabulary directly, we don't need to adapt
>> >  # the layer - the vocabulary is already set. The vocabulary contains the
# padding token ('') and OOV token ('[UNK]') as well as the passed tokens.
>> >
>> > vectorize_layer.get_vocabulary()
['', '[UNK]', 'earth', 'wind', 'and', 'fire']


In [ ]:
# import string
# import re

# # Having looked at our data above, we see that the raw text contains HTML break
# # tags of the form '<br />'. These tags will not be removed by the default
# # standardizer (which doesn't strip HTML). Because of this, we will need to
# # create a custom standardization function.


# def custom_standardization(input_data):
#   lowercase = tf.strings.lower(input_data)
#   stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
#   return tf.strings.regex_replace(stripped_html,
#                                   '[%s]' % re.escape(string.punctuation),
#                                   '')


In [ ]:


from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Model constants.
max_features = 20000
embedding_dim = 128

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=500)

# Now that the vocab layer has been created, call `adapt` on the text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.
vectorize_layer.adapt(text_dataset.batch(64))

# Next, let's build a model.





# --------------------------------------------------
# tf.keras.layers.TextVectorization(
#     max_tokens=None,
#     standardize='lower_and_strip_punctuation',
#     split='whitespace',
#     ngrams=None,
#     output_mode='int',
#     output_sequence_length=None,
#     pad_to_max_tokens=False,
#     vocabulary=None,
#     idf_weights=None,
#     sparse=False,
#     ragged=False,
#     encoding='utf-8',
#     **kwargs
# )

# #--------------------------------------------------

# tf.keras.layers.experimental.preprocessing.TextVectorization(
#     max_tokens=None,
#     standardize="lower_and_strip_punctuation",
#     split="whitespace",
#     ngrams=None,
#     output_mode="int",
#     output_sequence_length=None,
#     pad_to_max_tokens=False,
#     vocabulary=None,
#     **kwargs
# )



NameError: name 'TextVectorization' is not defined

In [ ]:
# file_object = open("filename", "mode")


# First pip install random-word

f= open("guru99.csv","w+")
for i in range(10):
    for i in range(20):
        f.write("This is line %d\r\n" % (i+1))
f.close()
    
 

In [ ]:
import urllib2

word_site = "https://www.mit.edu/~ecprice/wordlist.10000"


response = urllib2.urlopen(word_site)
txt = response.read()
WORDS = txt.splitlines()

ModuleNotFoundError: No module named 'urllib2'